# Genetic Algorithm from Scratch

### Kiana Ghamsari 400222079

In [1]:
import random

In [2]:
# inputs
def get_inputs():
    num_slots = int(input("Enter the number of slots in the equation: "))
    numbers = list(map(int, input("Enter the list of numbers (space-separated): ").split()))
    operators = input("Enter the list of operators (e.g., + - * %): ").split()
    target = int(input("Enter the target number (right-hand side of the equation): "))
    return {'num_slots': num_slots,
            'numbers': numbers,
            'operators': operators,
            'target': target}
    # return num_slots, numbers, operators, target

In [3]:
inputs = get_inputs()

In [4]:
print(inputs)

{'num_slots': 21, 'numbers': [1, 2, 3, 4, 5, 6, 7, 8], 'operators': ['+', '-', '*'], 'target': 18019}


In [5]:
print(inputs['numbers'])

[1, 2, 3, 4, 5, 6, 7, 8]


In [6]:
print(len(inputs['numbers']))

8


In [7]:
num_slots = inputs['num_slots']
gens_numbers = inputs['numbers']
gens_operators = inputs['operators']
target = inputs['target']

num_operators = num_slots//2
num_numbers = num_operators + 1

In [8]:
print(num_slots)
print(gens_numbers)
print(gens_operators)
print(target)

21
[1, 2, 3, 4, 5, 6, 7, 8]
['+', '-', '*']
18019


In [9]:
# create chromosome
def create_chromosome():
    chromosome = []
    for _ in range(num_operators):
        chromosome.append(random.choice(gens_numbers))
        chromosome.append(random.choice(gens_operators))
    chromosome.append(random.choice(gens_numbers))
    return chromosome

In [10]:
# Sample Chromosome
chromosome = create_chromosome()
print('Sample Chromosome:', chromosome)

Sample Chromosome: [5, '*', 4, '+', 2, '*', 3, '*', 3, '-', 5, '+', 3, '+', 3, '+', 8, '*', 6, '-', 8]


In [11]:
# Create Initial Population
def create_initial_population(pop_size):
    return [create_chromosome() for _ in range(pop_size)]

In [12]:
# Sample Initial Population
population = create_initial_population(10)
print("Initial Population:", population)

Initial Population: [[5, '-', 6, '+', 4, '*', 1, '-', 7, '*', 3, '+', 6, '*', 3, '*', 4, '+', 1, '-', 6], [2, '+', 7, '*', 7, '-', 3, '-', 2, '-', 8, '-', 8, '-', 4, '*', 1, '-', 6, '*', 8], [3, '-', 7, '*', 6, '+', 2, '+', 5, '+', 2, '+', 3, '*', 6, '+', 4, '-', 5, '*', 6], [1, '*', 6, '+', 4, '-', 1, '+', 8, '*', 8, '+', 1, '+', 2, '*', 6, '*', 6, '+', 7], [7, '*', 3, '-', 1, '*', 3, '-', 4, '-', 4, '*', 8, '+', 5, '+', 6, '-', 4, '*', 6], [4, '-', 5, '+', 5, '-', 8, '*', 3, '-', 5, '+', 5, '*', 8, '+', 1, '-', 4, '-', 5], [4, '-', 7, '-', 2, '*', 8, '*', 7, '*', 7, '+', 3, '-', 5, '+', 5, '+', 8, '*', 4], [4, '-', 1, '*', 4, '*', 1, '*', 1, '*', 6, '-', 7, '-', 3, '-', 6, '-', 6, '*', 7], [6, '-', 5, '-', 8, '-', 8, '+', 4, '+', 6, '*', 1, '*', 4, '*', 8, '-', 2, '-', 8], [6, '-', 7, '+', 8, '*', 3, '*', 6, '-', 5, '-', 8, '*', 8, '+', 5, '*', 6, '+', 7]]


In [13]:
# Fitness Function
def fitness_function(chromosome, target):
    try:
        # Evaluate expression value
        expression = ''.join(map(str, chromosome))
        # print("Expression:", expression)
        result = eval(expression)
        # print("result:", expression, "=", result)
        # print("target:", target)
        # print(f"|{target} - {result}| =", abs(target - result))
        return abs(target - result)
    except ZeroDivisionError:
        return float('inf')

In [18]:
# test fitness function
print(fitness_function(chromosome, target))

17940


In [19]:
# Test Fitness Function
def test_fitness_function(chromosome, target):
    try:
        # Evaluate expression value
        expression = ''.join(map(str, chromosome))
        print("Expression:", expression)
        result = eval(expression)
        print("result:", expression, "=", result)
        print("target:", target)
        print(f"|{target} - {result}| =", abs(target - result))
        return abs(target - result)
    except ZeroDivisionError:
        return float('inf')

In [ ]:
# test how the fitness function works
print(test_fitness_function(chromosome, target))

Expression: 5*4+2*3*3-5+3+3+8*6-8
result: 5*4+2*3*3-5+3+3+8*6-8 = 79
target: 18019
|18019 - 79| = 17940
17940


In [24]:
# Two-Point Crossover Between Two Chromosomes
def two_point_crossover(parent1, parent2):
    point1 = random.randint(0, len(parent1) - 1)
    point2 = random.randint(point1 + 1, len(parent1))
    
    child = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    return child

In [25]:
# test two-point crossover
print(two_point_crossover(population[0], population[1]))

[5, '-', 6, '+', 7, '-', 3, '-', 2, '-', 8, '-', 8, '-', 4, '*', 4, '+', 1, '-', 6]


In [26]:
# Chromosomal Mutation
def mutate(chromosome, mutation_rate=0.1):
    if random.random() < mutation_rate:
        mutation_index = random.randint(0, len(chromosome) - 1)
        if mutation_index % 2 == 0: # if number
            chromosome[mutation_index] = random.choice(gens_numbers)
        else: # if operator
            chromosome[mutation_index] = random.choice(gens_operators)
    return chromosome

#### What is elitism in genetic algorithms?
* A strategy in evolutionary algorithms where the best one or more solutions, called the elites, in each generation with the best fitness values, are inserted into the next, without undergoing any change.

#### Elitism Selection
* This method ensures that good genes are not destroyed due to crossover and mutation.

In [27]:
def select_elites(population, fitness_scores, elite_ratio=0.1):
    num_elites = int(len(population) * elite_ratio)
    sorted_population = [chromosome for chromosome, _ in sorted(fitness_scores, key=lambda x: x[1])]
    elites = sorted_population[:num_elites]
    return elites

In [60]:
def genetic_algorithm(target, pop_size=50, generations=100, mutation_rate=0.1, elite_ratio=0.1):
    population = create_initial_population(pop_size)
    
    for generation in range(generations):
        fitness_scores = [(chromosome, fitness_function(chromosome, target)) for chromosome in population]
        fitness_scores.sort(key=lambda x: x[1])
        
        best_chromosome, best_score = fitness_scores[0]
        print(f"Generation {generation}: Best Chromosome = {best_chromosome}, Fitness = {best_score}")
        
        if best_score == 0:
            break
        
        elites = select_elites(population, fitness_scores, elite_ratio)
        
        next_generation = elites
        while len(next_generation) < pop_size:
            parent1, parent2 = random.choices(population, k=2)
            child = two_point_crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            next_generation.append(child)
        
        population = next_generation
        
    return ''.join(map(str, best_chromosome))

In [84]:
print(genetic_algorithm(target, 50, 100, 0.1, 0.1))

Generation 0: Best Chromosome = [3, '+', 6, '*', 6, '*', 6, '*', 3, '+', 6, '-', 3, '-', 1, '*', 8, '-', 6, '-', 2], Fitness = 17381
Generation 1: Best Chromosome = [6, '*', 7, '*', 5, '*', 3, '+', 6, '*', 4, '*', 6, '*', 2, '-', 1, '+', 2, '-', 6], Fitness = 17106
Generation 2: Best Chromosome = [6, '*', 7, '*', 5, '*', 3, '+', 6, '*', 4, '*', 6, '*', 2, '-', 1, '+', 2, '-', 4], Fitness = 17104
Generation 3: Best Chromosome = [6, '*', 7, '*', 3, '*', 6, '*', 8, '+', 5, '+', 4, '*', 2, '-', 1, '+', 2, '-', 6], Fitness = 11963
Generation 4: Best Chromosome = [6, '*', 7, '*', 3, '*', 6, '*', 8, '+', 5, '+', 4, '*', 2, '-', 1, '+', 2, '-', 6], Fitness = 11963
Generation 5: Best Chromosome = [6, '*', 7, '*', 3, '*', 6, '*', 8, '-', 5, '+', 8, '*', 6, '*', 8, '+', 4, '-', 6], Fitness = 11594
Generation 6: Best Chromosome = [3, '-', 8, '*', 4, '+', 2, '+', 8, '*', 5, '*', 7, '*', 6, '*', 1, '*', 7, '+', 3], Fitness = 6283
Generation 7: Best Chromosome = [3, '-', 8, '*', 4, '+', 2, '+', 8, '*

## <p align="right"> بخش شش: سوالات </p>

<div dir="auto">

   #### 1. جمعیت اولیه بسیار کم یا بسیار زیاد چه مشکالتی بوجود می آورد؟
</div>


<div dir="auto">

  - **جمعیت بسیار کم:**
   
    1. تنوع ژنتیکی کم است و کروموزوم‌ها به سرعت شبیه به هم می‌شوند.

    2. احتمال گیر افتادن در بهینه محلی زیاد است.

    3. فضای جستجوی مسئله به خوبی پوشش داده نمی‌شود.

    4. در صورت وجود یک جمعیت کوچک، احتمال یافتن جواب مناسب کاهش می‌یابد.

       - کیفیت کلی الگوریتم کاهش می‌یابد.

  - **جمعیت بسیار زیاد:**

    1. زمان محاسبات بالا: ارزیابی Fitness و اجرای کراس‌اور و موتیشن برای جمعیت بزرگ زمان بیشتری نیاز دارد.
    
    2. استفاده از منابع محاسباتی (RAM و CPU) بیشتر می‌شود.

    3. پس از یک مقدار مشخص، افزایش جمعیت دیگر تأثیر معناداری روی کیفیت جواب ندارد.

       - الگوریتم ممکن است کند عمل کند، حتی اگر جواب درست را پیدا کند.

</div>

---

<div dir="auto">

   #### 2. اگر تعداد جمعیت در هر دوره افزایش یابد، چه تاثیری بر دقت و سرعت الگوریتم میگذارد؟
</div>



<div dir="auto">

  - تاثیر بر دقت:

    - فضای جستجو بهتر پوشش داده می‌شود.
    - افزایش جمعیت ممکن است تنوع بیشتری ایجاد کرده و احتمال یافتن جواب بهینه را افزایش دهد.
    - اما پس از رسیدن به یک حد مشخص، افزایش جمعیت تأثیر کمتری بر دقت خواهد داشت.

  - تاثیر بر سرعت:
  
    - افزایش جمعیت زمان اجرای الگوریتم را به دلیل محاسبات بیشتر (برای ارزیابی Fitness، کراس‌اور، و موتیشن) افزایش می‌دهد.

  - راهکار پیشنهادی:

    - افزایش تدریجی جمعیت و بررسی تأثیر آن می‌تواند به یافتن مقدار بهینه کمک کند.

    - استفاده از جمعیت پویا (Dynamic Population) می‌تواند مناسب باشد: در ابتدا جمعیت کوچک باشد و به تدریج افزایش یابد.

    
    - مثلاً:
    



</div>


  ```python

  if no_improvement_counter > 10:  # اگر 10 نسل بدون بهبود باشد

      pop_size += 10  # افزایش تعداد جمعیت
      
  ```

  ---

<div dir="auto">

   #### 3. تاثیر هر یک از عملیاتهای crossover و mutation را بیان و مقایسه کنید. آیا میتوان فقط یکی از آنها را استفاده کرد و چرا؟
</div>


<div align="right">

   ##### :Crossover
</div>

<div dir="auto">


   - **نقش**:
      - ترکیب اطلاعات دو کروموزوم برای ایجاد فرزندان جدید.
    
   - **تاثیر**:

      - بهترین ویژگی‌های والدین را به فرزندان منتقل می‌کند.
      - باعث بهبود سریع‌تر جمعیت و نزدیک شدن به جواب می‌شود.

   - **نقاط ضعف:**
   
      - تنوع کمی ایجاد می‌کند و ممکن است باعث یکنواختی جمعیت شود.



   ##### Mutation:

   - **نقش**:
      - تغییر تصادفی در ژن‌های کروموزوم.

   - **تاثیر**:

      - تنوع را در جمعیت حفظ می‌کند و از گیر افتادن در بهینه محلی جلوگیری می‌کند.
      - کمک می‌کند راه‌حل‌های جدید کشف شوند.

   -  **نقاط ضعف:**

      - اگر نرخ موتیشن زیاد باشد، ممکن است ساختار کروموزوم تخریب شود.

      

   ##### آیا می‌توان فقط یکی را استفاده کرد؟
   
   -  **استفاده تنها از Crossover**:
    باعث می‌شود تنوع کاهش یابد و الگوریتم به‌راحتی در نقاط بهینه محلی گیر کند.

   - **استفاده تنها از Mutation:** 
   پیشرفت بسیار کندی خواهد داشت، زیرا تغییرات تصادفی معمولاً زمان بیشتری برای رسیدن به جواب نیاز دارند
   و الگوریتم به یک فرآیند تصادفی شبیه می‌شود و یادگیری از نسل‌های قبلی محدود می‌شود.
   
   - **نتیجه:**
    ترکیب این دو روش ضروری است؛ زیرا Crossover برای بهره‌برداری از ویژگی‌های خوب و Mutation برای ایجاد تنوع استفاده می‌شود.

   ---


</div>

<div dir="auto">

   #### 4. بنظر شما چه راهکارهایی برای سریعتر به جواب رسیدن در این مسئله خاص وجود دارد؟
</div>


<div dir="auto">
   

   
   1. **حفظ نخبگان (Elitism):**
      - بهترین کروموزوم‌ها را مستقیماً به نسل بعد منتقل کنید (که باعث افزایش سرعت همگرایی می‌شود).
      - تضمین می‌کند که بهترین راه‌حل‌ها از بین نمی‌روند.

   2. **تنظیم نرخ موتیشن:**
      - استفاده از نرخ موتیشن متعادل (مثلاً `0.2`) برای افزایش تنوع بدون تخریب ساختار.

   3. **کاهش جمعیت در نسل‌های بعدی:**
      - اگر جمعیت خیلی بزرگ است، پس از چند نسل موفق، می‌توانید اندازه جمعیت را کاهش دهید.

   4. **انتخاب والدین تصادفی‌تر:**
      - از روش چرخ رولت یا انتخاب تورنمنت برای افزایش تنوع ژنتیکی استفاده کنید.

   5. **افزایش اندازه جمعیت اولیه:**
      - جمعیت اولیه متنوع‌تر شانس بیشتری برای یافتن جواب درست ایجاد می‌کند.

   6. **افزایش نرخ موتیشن:**
      - اگر جمعیت یکنواخت شده، نرخ موتیشن را افزایش دهید (مثلاً `0.3`).

---

</div>

<div dir="auto">

   #### 5. با وجود استفاده از این روش ها باز هم ممکن است که کروموزوم ها پس از چند مرحله دیگر تغییر نکند. دلیل این اتفاق و مشکلاتی که بوجود می آورد را شرح دهید. برای حل آن چه پیشنهادی میدهید؟ (راه حل های خود را امتحان و بهترین آنها را در پروژه خود پیاده سازی کنید)


   
   ##### دلیل:

   - یکنواختی جمعیت: اگر تنوع کروموزوم‌ها کاهش یابد، عملیات کراس‌اور تغییر قابل توجهی ایجاد نمی‌کند.

   - نرخ پایین موتیشن: تغییرات تصادفی کافی ایجاد نمی‌شود.

   - گیر افتادن در بهینه محلی: الگوریتم نمی‌تواند راه‌حل‌های جدید کشف کند.

   ##### مشکلات:

   - الگوریتم دیگر پیشرفتی ندارد.
   
   - زمان محاسبات بیهوده صرف می‌شود.

   - احتمال یافتن جواب درست کاهش می‌یابد.


   
   ##### پیشنهاد‌ها:

   1. **افزایش نرخ موتیشن**: تنوع بیشتری در کروموزوم‌ها ایجاد می‌کند.

   2. **Restart جمعیت**: اگر تنوع کاهش یافت، کل جمعیت را مجدداً به‌صورت تصادفی تولید کنید.
   
   3. **استفاده از چندین جمعیت (Multiple Populations)**: جمعیت‌های موازی که با هم تعامل دارند می‌توانند از یکنواختی جلوگیری کنند.

   4. **افزایش تصادفی جمعیت**: کروموزوم‌های تصادفی جدید به جمعیت اضافه کنید.



</div>

```python
if no_improvement_counter > 10:
    population.append(create_chromosome())
```



---

<div dir="auto">

   #### 6. چه راه حلی برای تمام شدن برنامه در صورتی که مسئله جواب نداشته باشد پیشنهاد میشود؟



  1. **محدود کردن تعداد نسل‌ها:**

     - یک حداکثر تعداد نسل تعریف کنید (مثلاً 100):

</div>

    
```python
if generation == max_generations:
    print("No solution found after 100 generations.")
    break
```

<div dir="auto">

  2. **شرط توقف بر اساس عدم بهبود:**

     - اگر برای تعداد مشخصی از نسل‌ها (مثلاً 20 نسل) هیچ بهبودی حاصل نشد، برنامه متوقف شود.

</div>


```python
if no_improvement_counter > 20:
    print("No improvement for 20 generations, stopping.")
    break
```

<div dir="auto">

  3. **محاسبه خطای حداقلی:**

     - حداقلی از Fitness که به عنوان جواب قابل قبول است را تعیین کنید. اگر Fitness به این حد برسد، الگوریتم متوقف می‌شود.


</div>


```python
if best_fitness < threshold:
    print("Closest solution found:", best_chromosome)
    break
```

<div dir="auto">

   4. **بازگشت بهترین جواب ممکن:**

      - اگر جواب دقیق پیدا نشود، الگوریتم بهترین کروموزوم تولیدشده را بازگرداند.

</div>



</div>